In [1]:
# ~~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ constantes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
limupper = 0.6; # GB, for whole file segmentation in uInt8
limite = 15; # maximum percentage of segment that can be saturated
HV = 2500; # upper voltage limit in μV
lV = -2500; # lower voltage limit in μV
method = 1; # method to repair saturation ( 1, 2 or 3 )
ΔT = 400; # ms for the temporary variation
window = 5; # ms for event detection
bit = 1; # window sliding milliseconds
distance = 0.3; # ms maximum distance between events to be discarded
thr = -60; # µV static threshold
mid = 0.6; # Value for plotting the metrics for ACD
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [2]:
# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ paquetes ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
using JLD
using Plots
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [3]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHBRWS = "/run/media/LBitn/Data/Medula/28-02-2022/BRWs";
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [4]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHFunctions = 
    "/home/LBitn/Dropbox/git-repos/Hippocampus-HDMEA/RomeroI-notebooks/ZapfeModifications";
push!( LOAD_PATH, PATHFunctions );
using MeaTools
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [6]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILESBRW = SearchDir( PATHBRWS, ".brw" ); 
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [9]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@time for F = 2:4
    FILEBRW = FILESBRW[ F ];
    Variables, FILEPATHS, FILEVARS = GetVarsHDF5( FILEBRW );
    PATHS = load( FILEPATHS )[ "PATHS" ];
    PATHMAIN = PATHS[ "PATHMAIN" ];
    PATHRAW = joinpath( PATHMAIN, "Segments" ); mkpath( PATHRAW );
    PATHMAIN = PATHS[ "PATHMAIN" ];
    PATHINFO = PATHS[ "PATHINFO" ];
    PATHFIGURES = joinpath( PATHMAIN, "Figures" ); mkpath( PATHFIGURES );
    PATHFIGSGENERAL = joinpath( PATHFIGURES, "General" ); mkpath( PATHFIGSGENERAL );
    PATHVOLT = joinpath( PATHMAIN, "Voltage" ); mkpath( PATHVOLT );
    PATHFILTERS = joinpath( PATHMAIN, "Filters" ); mkpath( PATHFILTERS );
    parameters = Dict(
                "window"    => Ms2Frs( window, Variables ),
                "bit"       => Ms2Frs( bit, Variables ),
                "distance"  => Ms2Frs( distance, Variables ),
                "ΔT"        => Ms2Frs( ΔT, Variables );
            );
    σ = ChunkSizeSpace( Variables, limupper ); # Number of segments to cut
    n0s = length( string( σ ) );
    n = σ;
    BIN = OneSegment( Variables, n, σ );
    BINNAME = string( "S", lpad( n, n0s, "0" ) );
    BINRAW = Digital2Analogue( Variables, BIN );
    BINRAW, _, _, _ = FixSaturation( 
        BINRAW, limite, HV, lV, method );
    save( joinpath( PATHVOLT, string( BINNAME, ".jld" ) ), "RAW", Float16.( BINRAW ) );
    S = PFT_mt( Variables, BINRAW );
    Ws, AE, BINMUA, BINLFP, metrics = AllWs( Variables, BINRAW, parameters, thr );
    save( joinpath( 
        PATHFILTERS, string( BINNAME, ".jld") ), "LFP", BINLFP, "MUA", BINMUA );
    BINMUA = [ ]; BINLFP = [ ];
    W0NTotal, PF = W0Plots( Ws, BINRAW, mid, metrics );
    savefig( PF, joinpath( PATHFIGSGENERAL, string( BINNAME, "W0NPlots.png" ) ) );
    savefig( PF, joinpath( PATHFIGSGENERAL, string( BINNAME, "W0NPlots.svg" ) ) );
    W = Dict(
        "DirectWeights"  => Ws,
        "DetectedEvents" => AE,
        "labels"         => metrics,
        "LogWeights"     => W0NTotal
    );
    save( joinpath( 
        PATHINFO, string( BINNAME, "_Analitics.jld" ) ), "Spectrum", S, "Weights", W );
    S = [ ]; Ws = [ ]; AE = [ ]; metrics = [ ]; W0NTotal = [ ]; W = [ ];
    println( BINNAME, " Done" );
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

You are now working on the new main path: /run/media/LBitn/Data/Medula/28-02-2022/Phase_02
With the file: 
Phase_02.brw : BRW-File Level3 - 3Brain RaW data file for high resolution MEA platform, HDF5-format - Created with BrainWave v.4.5.8089.17174 on date Monday, February 28, 2022 HDF5 file size: 24.533904634416103 GB
45 segments of 0.545 GB and 4.0 seconds each
92 channels repaired with the method: 
Random values taken from the same channel
25 discarted channels.
S45 Done
You are now working on the new main path: /run/media/LBitn/Data/Medula/28-02-2022/Phase_06
With the file: 
Phase_06.brw : BRW-File Level3 - 3Brain RaW data file for high resolution MEA platform, HDF5-format - Created with BrainWave v.4.5.8089.17174 on date Monday, February 28, 2022 HDF5 file size: 24.533904641866684 GB
45 segments of 0.545 GB and 4.0 seconds each
19 channels repaired with the method: 
Random values taken from the same channel
3 discarted channels.
S45 Done
You are now working on the new main path: /

In [11]:
Phase_01 = load( 
    "/run/media/LBitn/Data/Medula/28-02-2022/Phase_01/Info/S45_Analitics.jld" );
Phase_02 = load( 
    "/run/media/LBitn/Data/Medula/28-02-2022/Phase_02/Info/S45_Analitics.jld" );
Phase_06 = load( 
    "/run/media/LBitn/Data/Medula/28-02-2022/Phase_06/Info/S45_Analitics.jld" );
Phase_08 = load( 
    "/run/media/LBitn/Data/Medula/28-02-2022/Phase_08/Info/S45_Analitics.jld" );

In [16]:
aux = Phase_01["Weights"]["DetectedEvents"][1];

In [21]:
using StatsBase
C = countmap( vcat( aux... ) );

In [22]:
K = collect( keys( C ) );
V = collect( values( C ) );

In [30]:
testing = K[ findall( V .> 1 ) ];

In [36]:
ch_r = [ ]
frm_r = [ ]

for i in 1:length( aux )
    for j in 1:length( testing )
        x = findall( aux[ i ] .== testing[ j ] )
        if !isempty( x )
            push!( ok, [ i, testing[ j ], x ] )
        end
    end
end

205277-element Vector{Any}:
 Any[2, 63424, [54]]
 Any[2, 17830, [28]]
 Any[2, 16049, [24]]
 Any[2, 22133, [39]]
 Any[2, 22100, [38]]
 Any[2, 3402, [4]]
 Any[2, 17809, [27]]
 Any[2, 66852, [58]]
 Any[2, 59978, [44]]
 Any[2, 13099, [9]]
 Any[2, 13073, [8]]
 Any[2, 13684, [16]]
 Any[2, 16073, [25]]
 ⋮
 Any[4096, 24976, [47]]
 Any[4096, 67989, [116]]
 Any[4096, 13680, [29]]
 Any[4096, 13113, [26]]
 Any[4096, 10785, [8]]
 Any[4096, 16035, [33]]
 Any[4096, 46189, [74]]
 Any[4096, 39348, [67]]
 Any[4096, 67980, [115]]
 Any[4096, 24949, [45]]
 Any[4096, 53588, [77]]
 Any[4096, 32352, [61]]